# Interacting with the Blackboard

In [31]:
import urllib, requests, json
bb_url = 'http://localhost:9000/blackboard/api'

In [32]:
query = {"type": "query", "name": "A simple blackboard example","term": "asthma"}
json.dumps(query)
req = requests.post(bb_url, json=query)
req.status_code

200

Having posted the query, we then retrieve the resultant knowledge graph is JSON form.

In [33]:
kg = requests.get(bb_url)
kg.json()

[{u'edgeCount': 0,
  u'edges': [],
  u'id': 1,
  u'name': u'A simple blackboard example',
  u'nodeCount': 1,
  u'nodes': [{u'degree': 0,
    u'id': 2,
    u'inDegree': 0,
    u'name': u'A simple blackboard example',
    u'outDegree': 0,
    u'tags': [u'KG:1', u'query', u'KQuery'],
    u'term': u'asthma',
    u'type': u'query'}],
  u'type': u'kgraph'}]

We then update the KG using the Pharos KS (not this needs to be a `PUT` request)

In [34]:
bb_id = kg.json()[0]['id']
req = requests.put(bb_url+'/'+str(bb_id)+"/ks.pharos")
req.status_code

200

Now we retrieve the current state of the KG


In [36]:
req = requests.get(bb_url)
req.json()

[{u'edgeCount': 60,
  u'edges': [{u'directed': False,
    u'id': 0,
    u'source': 2,
    u'target': 4,
    u'type': u'assertion'},
   {u'directed': False,
    u'id': 1,
    u'source': 2,
    u'target': 3,
    u'type': u'assertion'},
   {u'directed': False,
    u'id': 2,
    u'source': 2,
    u'target': 5,
    u'type': u'assertion'},
   {u'directed': False,
    u'id': 3,
    u'source': 2,
    u'target': 6,
    u'type': u'assertion'},
   {u'directed': False,
    u'id': 4,
    u'source': 2,
    u'target': 7,
    u'type': u'assertion'},
   {u'directed': False,
    u'id': 5,
    u'source': 2,
    u'target': 8,
    u'type': u'assertion'},
   {u'directed': False,
    u'id': 6,
    u'source': 2,
    u'target': 9,
    u'type': u'assertion'},
   {u'directed': False,
    u'id': 7,
    u'source': 2,
    u'target': 10,
    u'type': u'assertion'},
   {u'directed': False,
    u'id': 8,
    u'source': 2,
    u'target': 11,
    u'type': u'assertion'},
   {u'directed': False,
    u'id': 9,
    u'source

As noted in the [documentation](https://spotlite.nih.gov/ncats/blackboard) we can rerun the Pharos KS to add more nodes and edges. The first call, above, added nodes that were related to the query node (and the associated edges). When we call it again, it will add nodes that are related to the nodes added in the previous run (i.e., 2nd neighbors of the query node)

In [37]:
bb_id = kg.json()[0]['id']
req = requests.put(bb_url+'/'+str(bb_id)+"/ks.pharos")

In [38]:
req = requests.get(bb_url)
ofile = open('kg.json', 'w') ## for reference
ofile.write(req.text)
ofile.close()

Currently the knowledge graph is stored in a [Neo4J](https://neo4j.com/) instance. We can interact with this assuming you have a Neo4J client running and pointed to `blackboard.db` that was created by the blackboard API.

Alternatively, we can parse the JSON representation of the knowledge graph into an `igraph` object and then visualize it using the Cytoscape [extension](https://github.com/cytoscape/jupyter-cytoscape) for Jupyter notebooks. (This is most easily installed if you're using the Anaconda distribution of Python). 

First we make a function to go from the Knowlegde Graph JSON to an `igraph` object

In [42]:
from igraph import *

def kg2ig(o):
    kg = o[0]
    if kg['type'] != 'kgraph':
        raise Exception, "Must provide a JSON kgraph"

    g = Graph(directed=False)

    nodes = kg['nodes']
    for node in nodes:
        d = {}
        for key in node.keys():
            if key in ['inDegree','outDegree','degree', 'id']: continue
            key = key.encode('ascii')
            d[key] = node[key]
        g.add_vertex(**d)

    edges = kg['edges']
    for edge in edges:
        s = filter(lambda x: x['id'] == edge['source'], nodes)
        t = filter(lambda x: x['id'] == edge['target'], nodes)
        if len(s) == 1 and len(t) == 1:
            g.add_edge(s[0]['name'],t[0]['name'], type=edge['type'])
        
    return g

We then use this to get the object and then display it via the Cytoscape extension

In [45]:
g = kg2ig(json.loads(req.text))
print g.summary()

IGRAPH UN-T 177 191 -- 
+ attr: family (v), name (v), synonyms (v), tags (v), term (v), type (v), uri (v), type (e)


In [52]:
from cyjs import *

cy = cyjs()
display(cy)

cy.deleteGraph()
cy.addGraph(g)
kkLayout = g.layout("kk")
cy.setPosition(kkLayout)
cy.fit(10)
cy.setHeight(500)

